<a href="https://colab.research.google.com/github/Neo-glitch/t.f-2.0-practice/blob/master/Transfer_Learning_pratice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate \
      https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2022-03-28 23:40:55--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.125.128, 142.250.136.128, 142.250.148.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   213MB/s    in 0.3s    

2022-03-28 23:40:56 (213 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
tf.config.run_functions_eagerly(True)

import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline


###Unziping Dataset###

In [ ]:
dataset_path = "./cats_and_dogs_filtered.zip"

zip_object = zipfile.ZipFile(file = dataset_path, mode ="r")
zip_object.extractall("./")

zip_object.close()

In [ ]:
dataset_path_new = "./cats_and_dogs_filtered/"

train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [ ]:
IMG_SHAPE = (128, 128, 3)

base_model = keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top = False, weights="imagenet")

base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [ ]:
base_model.trainable = False  # freeze model layers

In [ ]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [ ]:
# flatten the output
global_average_layer = keras.layers.GlobalAveragePooling2D()(base_model.output)

global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d_1')>

In [ ]:
prediction_layer = keras.layers.Dense(units = 1, activation = "sigmoid")(global_average_layer)

In [ ]:
model = keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']         

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

train_gen = data_gen_train.flow_from_directory(train_dir, target_size = (128, 128), batch_size=128, class_mode="binary")
valid_gen = data_gen_valid.flow_from_directory(validation_dir, target_size = (128, 128), batch_size = 128, class_mode ="binary")



Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
model.fit(train_gen, epochs=5, validation_data=valid_gen)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


16/16 [==============================] - 12s 775ms/step - loss: 0.7302 - accuracy: 0.5800 - val_loss: 0.6670 - val_accuracy: 0.6440
Epoch 2/5
16/16 [==============================] - 12s 749ms/step - loss: 0.6072 - accuracy: 0.6715 - val_loss: 0.5729 - val_accuracy: 0.7210
Epoch 3/5
16/16 [==============================] - 12s 749ms/step - loss: 0.5260 - accuracy: 0.7535 - val_loss: 0.5005 - val_accuracy: 0.7780
Epoch 4/5
16/16 [==============================] - 12s 730ms/step - loss: 0.4617 - accuracy: 0.8055 - val_loss: 0.4405 - val_accuracy: 0.8220
Epoch 5/5
16/16 [==============================] - 12s 739ms/step - loss: 0.4072 - accuracy: 0.8500 - val_loss: 0.3916 - val_accuracy: 0.8550


In [ ]:
model.evaluate_generator(valid_gen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


[0.3916342854499817, 0.8550000190734863]

###Fine Tuning###

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [ ]:
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_gen,  
                    epochs=5, 
                    validation_data=valid_gen)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


16/16 [==============================] - 12s 776ms/step - loss: 0.2002 - accuracy: 0.9170 - val_loss: 0.0887 - val_accuracy: 0.9680
Epoch 2/5
16/16 [==============================] - 12s 746ms/step - loss: 0.0328 - accuracy: 0.9960 - val_loss: 0.0961 - val_accuracy: 0.9670
Epoch 3/5
16/16 [==============================] - 12s 750ms/step - loss: 0.0120 - accuracy: 0.9995 - val_loss: 0.1217 - val_accuracy: 0.9610
Epoch 4/5
16/16 [==============================] - 12s 745ms/step - loss: 0.0066 - accuracy: 0.9995 - val_loss: 0.1552 - val_accuracy: 0.9600
Epoch 5/5
16/16 [==============================] - 12s 749ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.2042 - val_accuracy: 0.9550


In [ ]:
 model.evaluate_generator(valid_gen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


[0.20417025685310364, 0.9549999833106995]